<a href="https://colab.research.google.com/github/GiovanniBorrelli/Software-Requirements-to-User-Stories/blob/main/ConfrontReqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk rouge-score
!pip install bert-score

import nltk
nltk.download('wordnet')

import numpy as np
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from sklearn.feature_extraction.text import TfidfVectorizer
from bert_score import score as bert_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2c61b91889281fbdac088c575ceda2ff04ddb2d7076c4ec072ed1c8746832f6b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Carica le user stories dai file (assicurati che i file siano nel tuo ambiente Colab)
with open('llama_chainofthoughts_reduced10.txt', 'r') as f:
    llama_stories = [story.strip() for story in f.readlines()]

with open('gpt_zeroshot_red10cot.txt', 'r') as f:
    oracle_stories = [story.strip() for story in f.readlines()]

# Rimozione degli spazi bianchi iniziali e finali da ogni User Story
llama_stories = [story.strip() for story in llama_stories]
oracle_stories = [story.strip() for story in oracle_stories]

In [ ]:
# Funzione per calcolare BLEU score con smoothing
def calculate_bleu(oracle_stories, llama_stories):
    bleu_scores = []
    smoothie = SmoothingFunction().method4  # Usa un metodo di smoothing
    for ref, cand in zip(oracle_stories, llama_stories):
        bleu_scores.append(sentence_bleu([ref.split()], cand.split(), smoothing_function=smoothie))
    return sum(bleu_scores) / len(bleu_scores)

# Funzione per calcolare ROUGE score per tutte le User Stories
def calculate_rouge(oracle_stories, llama_stories):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, cand in zip(oracle_stories, llama_stories):
        scores = scorer.score(ref, cand)
        for metric in rouge_scores:
            rouge_scores[metric].append(scores[metric].fmeasure)
    # Formattazione a 4 cifre decimali
    return {metric: round(sum(rouge_scores[metric]) / len(rouge_scores[metric]), 4) for metric in rouge_scores}

# Funzione per calcolare METEOR score con tokenizzazione
def calculate_meteor(oracle_stories, llama_stories):
    meteor_scores = []
    for ref, cand in zip(oracle_stories, llama_stories):
        meteor_scores.append(meteor_score([ref.split()], cand.split()))
    return sum(meteor_scores) / len(meteor_scores)

# Funzione per calcolare BERTScore
def calculate_bertscore(oracle_stories, llama_stories):
    P, R, F1 = bert_score(llama_stories, oracle_stories, lang="en", rescale_with_baseline=True)
    return F1.mean().item()

# Calcolo delle metriche
bleu = calculate_bleu(oracle_stories, llama_stories)
rouge = calculate_rouge(oracle_stories, llama_stories)
meteor = calculate_meteor(oracle_stories, llama_stories)
bertscore = calculate_bertscore(oracle_stories, llama_stories)

# Stampa dei risultati
print(f'BLEU Score medio: {bleu:.4f}')
print(f'ROUGE Scores medi: {rouge}')
print(f'METEOR Score medio: {meteor:.4f}')
print(f'BERTScore medio: {bertscore:.4f}')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score medio: 0.1879
ROUGE Scores medi: {'rouge1': 0.7045, 'rouge2': 0.4241, 'rougeL': 0.5589}
METEOR Score medio: 0.5157
BERTScore medio: 0.6261
